# Code underpinning the draft paper ‘The future environmental impact of irrigated agriculture is grossly underestimated’ - Python version

This script starts from the results of the regressions with the *R* script to develop the computation of the irrigated area, on which uncertainty and sensitivity analysis are eventually performed. The rationale and the function of each block of code are briefly described.

## Import all the relevant libraries

Python scripts resort to widely used libraries for statistical analysis and operations *(SciPy and Numpy)*, data curation and processing *(Pandas)* and data visualisation *(Matplotlib and Seaborn)*, which need to be imported at the beginning of the script.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sobol_seq

## Define a few functions to be used in the computation

Here convenient functions to be used in the script are defined. *Y* is the model output, the irrigated area. The trigger-indices function allow to select a particular row of the data frame (with the associated values of parameters) dependent on the value of the trigger sampled.

In [ ]:
def Y(Y0,Beta,Epsilon,r,N0,Gamma,Year0):
    return Y0+10**(a+b*Beta+Epsilon)*Beta*r*(N0**(1-Gamma)+r*(2050-Year0)*(1-Gamma))**((Beta+Gamma-1)/
    (1-Gamma))

def create_dict(key, values):
    return dict(zip(key, values))

def trigger_index(t):
    return np.sum(t*[len(lu_pop)/len(Areas2),len(lu_pop)/(len(Areas2)*len(dataSets)),len(lu_pop)/(len(Areas2)*len(dataSets)*
len(Regression_Method)),len(lu_pop)/(len(Areas2)*len(dataSets)*len(Regression_Method)*len(Robustness)),1],axis=1).astype(int)

def trigger_index_water(t):
    return np.sum(t*[len(lu_water)/len(Areas2),len(lu_water)/(len(Areas2)*len(dataSets)),
                len(lu_water)/(len(Areas2)*len(dataSets)*len(Robustness)),1],axis=1).astype(int)
 

## Read the input parameters

The input parameters are retrieved from the figures available in the literature.

In [ ]:
irrArea = pd.read_excel('agricultural_area.xlsx',sheet_name='meier')

Areas2 = ['Africa','Americas','Asia','Europe']
dataSets = ['Aquastat','FAOSTAT','Meier.et.al.2018','Salmon.et.al.2015','Siebert.et.al.2013','Thenkabail.et.al.2009']
Regression_Method = ['OLS','SMA']
Robustness = ['YES','NO']
sl = [['OLS','NO'],['OLS','YES'],['SMA','NO'],['SMA','YES']]
Y0 = pd.DataFrame([irrArea[irrArea['Continent']==ar2].sum() for ar2 in Areas2], index=[ar2 for ar2 in Areas2])
Y0=Y0.iloc[:,3:]/1e6
Y0=Y0.reindex(sorted(Y0.columns), axis=1)

## Prepare the data set

The results of the regressions run with *R* are retrieved for further computations.

In [ ]:
lu_pop = pd.read_csv('lookup.pop.csv',header=0)
lu_pop = lu_pop.sort_values(by=['Continent','Dataset','Regression','Robust','Beta']).reset_index(drop=True)
lu_water = pd.read_csv('lookup.water.csv',header=0)
lu_water = lu_water.sort_values(by=['Continent','Dataset','Robust','Delta']).reset_index(drop=True)

## Set the lookup table

Minor format changes are performed in the interest of univocal attributions.

In [ ]:
lu_water = lu_water.rename(columns={'Continent':'Continent_w','Dataset':'Dataset_w','Regression':'Regression_w',
                                   'Robust':'Robust_w'})

## K setting the carrying capacity factor (cropland available)

A table for the carrying capacity is defined on the basis of the figures reported in *Zhang and Cai 2011* for the net and gross cropland available.
 

In [ ]:
K = pd.DataFrame([[7.14e8,8.85e8,6.68e8,4.78e8],[1.205e9,1.481e9,8.89e8,7.05e8]],columns=[ar2 for ar2 in Areas2],index=\
                    ['Net','Gross']).T
K = K/1.e6

## $W_a$ setting the water carrying capacity factor (water available)

Analogously, the table is set for the water availability $W_a$ as per section 3.8 of the supplementary material.

In [ ]:
Wa = pd.DataFrame([[3142,8851,15629,5261],[4714,13277,23553,7891]],columns=[ar2 for ar2 in Areas2],index=\
                    ['Lower','Upper']).T

## $\eta$ water available share for irrigation

The same goes for the $\eta$ parameter as per the following section 3.9 of the supplementary material.

In [ ]:
eta = pd.Series([0.2,0.5],index=['Lower','Upper'])

## Quasi-random matrix for the Monte Carlo simulations

A low-discrepancy quasi random matrix is defined for the sampling from the distributions of the input parameters *Sobol 2011*. This sequence is to be preferred over purely random approaches due to its better convergence properties.

In [ ]:
qM =sobol_seq.i4_sobol_generate(29,2**16)

## Building the triggers for the lookup table

The triggers for selecting the rows of the look-up table for the regression coefficients of the population and water plots are defined in the block below.

In [ ]:
continent_v = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, 2**16) 
for x in range(len(Areas2)))))

triggers = np.vstack((continent_v,np.tile((qM[:,0:4]*[6,2,2,10_000]).T,4))).astype(int)

triggers_w = np.vstack((continent_v,np.tile((qM[:,4:7]*[6,2,10_000]).T,4))).astype(int)

triggersB = np.vstack((continent_v,np.tile((qM[:,14:18]*[6,2,2,10_000]).T,4))).astype(int)

triggers_wB = np.vstack((continent_v,np.tile((qM[:,18:21]*[6,2,10_000]).T,4))).astype(int)

## Extract the coefficients for the *r* parameter

The coefficients are retrieved from the *R* code implementation also in this case.

In [ ]:
p3c = pd.read_csv('AB_r_.csv')
p4 = pd.concat([pd.Series(p3c[p3c.Continent==ar2].iloc[:2**17].sample(n=2**16,replace=False)) for ar2 in Areas2])
p4B = pd.concat([pd.Series(p3c[p3c.Continent==ar2].iloc[2**17:2*2**27].sample(n=2**16,replace=False)) for ar2 in Areas2])

## Define the coefficients for the alpha regression in terms of Beta

The coefficients used for describing the relation between the regression parameters $\alpha$ and $\beta$ are reported below.

In [ ]:
a = -0.488873
b = -1.245859
epsilon = 0.1581345

## Population initial figures

The data set with the population figures in the years 1999-2012 - the range of starting years of our simulation - is retrieved from the *United Nations* estimates.

In [ ]:
PPop = pd.read_csv('UN_PP1999-2012.csv')
del PPop['Variant']
PPop = PPop.rename(columns={'Country or Area':'Continent','Year(s)':'Year'})
PPop=PPop.pivot(index='Continent',columns='Year',values='Value')
PPop.loc['Americas']=PPop.loc['Latin America and the Caribbean']+PPop.loc['Northern America']
PPop = PPop.drop(['Latin America and the Caribbean','Northern America'])
PPop = PPop.sort_index()

Different combinations of triggers for the parameters $X_1, X_2, X_3, X_4, w_1, w_2, w_4$ are defined so as to perform the sensitivity analysis at a later stage.

In [ ]:
tr_As = [triggers] 
for tr in range(1,len(triggers)):
    tr_As.append(triggers.copy())
    tr_As[tr][tr]=triggersB[tr]

tr_As_w = [triggers_w] 
for tr in range(1,len(triggers_w)):
    tr_As_w.append(triggers_w.copy())
    tr_As_w[tr][tr]=triggers_wB[tr]
    
tr_Bs = [triggersB] 

tr_Bs_w = [triggers_wB] 
    
tIndexA = [trigger_index(tri.T) for tri in tr_As]
tIndexB = [trigger_index(triB.T) for triB in tr_Bs]

tIndexA_w = [trigger_index_water(tri.T) for tri in tr_As_w]
tIndexB_w = [trigger_index_water(triB.T) for triB in tr_Bs_w]

In [ ]:
sM_As = []
for itI,tI in enumerate(tIndexA):
    sM_As.append(lu_pop.copy())
    sM_As[itI]=sM_As[itI].reindex(tI)
    sM_As[itI]=sM_As[itI].reset_index(drop=True)
    
sM_Bs = []
for itIB,tIB in enumerate(tIndexB):
    sM_Bs.append(lu_pop.copy())
    sM_Bs[itIB]=sM_Bs[itIB].reindex(tIB)
    sM_Bs[itIB]=sM_Bs[itIB].reset_index(drop=True)
    
sM_As_w = []
for itI,tI in enumerate(tIndexA_w):
    sM_As_w.append(lu_water.copy())
    sM_As_w[itI]=sM_As_w[itI].reindex(tI)
    sM_As_w[itI]=sM_As_w[itI].reset_index(drop=True)
    sM_As_w[itI]=pd.concat([sM_As[0],sM_As_w[itI]],axis=1)
    
sM_Bs_w = []
for itIB,tIB in enumerate(tIndexB_w):
    sM_Bs_w.append(lu_water.copy())
    sM_Bs_w[itIB]=sM_Bs_w[itIB].reindex(tIB)
    sM_Bs_w[itIB]=sM_Bs_w[itIB].reset_index(drop=True)
    sM_Bs_w[itIB]=pd.concat([sM_Bs[0],sM_Bs_w[itIB]],axis=1)

sM_A = []    
for sma in sM_As:
    sM_A.append(pd.concat([sma,sM_As_w[0].iloc[:,6:]],axis=1))

sM_B = []    
for smb in sM_Bs:
    sM_B.append(pd.concat([smb,sM_Bs_w[0].iloc[:,6:]],axis=1))

sM_A.extend(sM_As_w[1:])
sM_B.extend(sM_Bs_w[1:])

## Sample from the parameters distribution

Samples are extracted from the distribution of all the model input parameters in order to calculate the model output as well as run the uncertainty and sensitivity analysis.

In [ ]:
for sm in sM_A:
    sm['r']=p4.values
    sm['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,7]-1),4)
    sm['Year0'] = np.tile((PPop.columns.min()+(PPop.columns.max()-PPop.columns.min()+1)*qM[:,8]).astype(int),4)
    sm['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,9]-1)+1,4)
    sm['Y0']=0
    sm['K']=0
    sm['Wa']=0
    sm['eta']=np.tile(eta['Lower']+qM[:,13]*(eta['Upper']-eta['Lower']),4)
    for ar2 in Areas2:
        sm.loc[sm['Continent']==ar2,'Y0']=Y0.loc[ar2].min()+qM[:,10]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        sm.loc[sm['Continent']==ar2,'K']=K['Net'].loc[ar2]+qM[:,11]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
        sm.loc[sm['Continent']==ar2,'Wa']=Wa['Lower'].loc[ar2]+qM[:,12]*(Wa['Upper'].loc[ar2]-Wa['Lower'].loc[ar2])
    sm['N0']=[PPop[smy][ar2] for ar2 in Areas2 for smy in sm['Year0'][:int(len(sm)/4)]]
        
for smB in sM_B:
    smB['r']=p4B.values
    smB['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,21]-1),4)
    smB['Year0'] = np.tile((PPop.columns.min()+(PPop.columns.max()-PPop.columns.min()+1)*qM[:,22]).astype(int),4)
    smB['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,23]-1)+1,4)
    smB['Y0']=0
    smB['K']=0
    smB['Wa']=0
    smB['eta']=np.tile(eta['Lower']+qM[:,27]*(eta['Upper']-eta['Lower']),4)
    for ar2 in Areas2:
        smB.loc[smB['Continent']==ar2,'Y0']=Y0.loc[ar2].min()+qM[:,24]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        smB.loc[smB['Continent']==ar2,'K']=K['Net'].loc[ar2]+qM[:,25]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
        smB.loc[smB['Continent']==ar2,'Wa']=Wa['Lower'].loc[ar2]+qM[:,26]*(Wa['Upper'].loc[ar2]-Wa['Lower'].loc[ar2])
    smB['N0']=[PPop[smy][ar2] for ar2 in Areas2 for smy in smB['Year0'][:int(len(smB)/4)]]

## Produce the scrambled matrices

The scrambled matrices used for the computation of the sensitivity indices as per equations *(23)* and *(24)* of the supplementary material are produced by replacing columns of matrix A with matrix B.

In [ ]:
variables2 = [['r'],['Y0'],['Epsilon'],['Year0','N0'],['Gamma'],['K'],['Wa'],['eta']]

variables = [['Dataset'],['Regression'],['Robust'],['X4'],['Dataset_w'],['Robust_w'],['w4'],['r'],['Y0'],['Epsilon'],['Year0','N0'],
             ['Gamma'],['K'],['Wa'],['eta']]

In [ ]:
for iv,v in enumerate(variables2):
    sM_A.append(sM_A[0].copy())
    sM_A[-1][v]=sM_B[0][v]

The output is then calculated and the figures overshooting the land or water constraints are capped to these latter.

In [ ]:
for sm in sM_A:
    sm['Y']=Y(sm.Y0,sm.Beta,sm.Epsilon,sm.r,sm.N0,sm.Gamma,sm.Year0)
    sm.loc[sm['Y']>sm.K,'Y'] = sm.K
    sm.loc[(sm.Wa*sm.eta<sm.Y**sm.Delta*10**sm.Phi)&((sm.Wa*sm.eta/10**sm.Phi)**(1/sm.Delta) > sm.Y0),'Y'] = \
    (sm.Wa*sm.eta/10**sm.Phi)**(1/sm.Delta)
    sm.loc[(sm.Wa*sm.eta<sm.Y**sm.Delta*10**sm.Phi)&((sm.Wa*sm.eta/10**sm.Phi)**(1/sm.Delta) < sm.Y0),'Y'] = \
    sm.Y0
        
for smB in sM_B:
    smB['Y']=Y(smB.Y0,smB.Beta,smB.Epsilon,smB.r,smB.N0,smB.Gamma,smB.Year0)
    smB.loc[smB['Y']>smB.K,'Y'] = smB.K
    smB.loc[(smB.Wa*smB.eta<smB.Y**smB.Delta*10**smB.Phi)&((smB.Wa*smB.eta/10**smB.Phi)**(1/smB.Delta) 
            > smB.Y0),'Y'] = (smB.Wa*smB.eta/10**smB.Phi)**(1/smB.Delta)
    smB.loc[(smB.Wa*smB.eta<smB.Y**smB.Delta*10**smB.Phi)&((smB.Wa*smB.eta/10**smB.Phi)**(1/smB.Delta)
            < smB.Y0),'Y'] = smB.Y0

Potential negative figures are ruled out.

In [ ]:
sM_A = [sm[sm['Y']>0] for sm in sM_A]

sM_B = [smB[smB['Y']>0] for smB in sM_B]

## Append the B matrix for the Sobol-indices computation

In [ ]:
sM_S = sM_A.copy()
sM_S.append(sM_B[0])

The indices across the pool of sample and scrambled matrices are then reduced to the smallest sub-set as to rule out potential orphans across matrices, i.e. rows absent in one or more matrices.

In [ ]:
inr = sM_S[0].index.intersection(sM_S[1].index).intersection(sM_S[2].index).intersection(sM_S[3].index).\
intersection(sM_S[4].index).intersection(sM_S[5].index).intersection(sM_S[6].index).intersection(sM_S[7].index).\
intersection(sM_S[8].index).intersection(sM_S[9].index).intersection(sM_S[10].index).intersection(sM_S[11].index).\
intersection(sM_S[12].index).intersection(sM_S[13].index).intersection(sM_S[14].index).intersection(sM_S[15].index).\
intersection(sM_S[16].index)

In [ ]:
sM_S = [sms.merge(pd.DataFrame(index=inr), left_index=True, right_index=True, how='right') for sms in sM_S]

## Assess the Global Irrigated Area in 2050

In [ ]:
worldIrrigatedArea = pd.concat([sM_A[0].Y[ia*int(len(sM_A[0].Y)/len(Areas2)):(ia+1)*int(len(sM_A[0].Y)/len(Areas2))].reset_index(drop=True) 
                                for ia in range(len(Areas2))],axis=1,ignore_index=True).sum(axis=1)

## Bootstrapping of the sensitivity indices

A large low-discrepancy sequence is used for bootstrapping the calculation as to obtain robust estimates. **Note** this block can take time to execute as the generation of the low-discrepancy sequence requires time.

## Seed for bootstrapping

In [ ]:
qMbS = (sobol_seq.i4_sobol_generate(1,4*1000*2**16)).flatten()

### Bootstrap sensitivity indices

The row of the input matrices are bootstrapped one thousand times. **Note** Please bear in mind this computation can take several hours on an average market PC in 2019.

In [ ]:
c_r = []
T_r = []
Sa_r = []
for re in range(1000):    
    seed = (qMbS[re*len(sM_S[0]):(re+1)*len(sM_S[0])]*len(sM_S[0])).astype(int)
    
    sM_BS = [smS.merge(pd.DataFrame(index=seed), left_index=True, right_index=True, how='right') for smS in sM_S]

    V = []
    Sa_V = []
    T_V = []
    
    for imb,mb in enumerate(sM_BS[1:-1]):
        Geo = []
        Sa_G = []
        T_G = []
        for ar2 in Areas2:
            Var = np.var(np.concatenate((sM_BS[-1]['Y'][sM_BS[-1]['Continent']==ar2].values,
                             sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2].values),axis=None),ddof=0)
            Sa_G.append(np.mean((mb['Y'][mb['Continent']==ar2].values-sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2].values)*\
                         sM_BS[-1]['Y'][sM_BS[-1]['Continent']==ar2].values)/Var)
            T_G.append(0.5*(np.mean((mb['Y'][mb['Continent']==ar2].values-sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2].values)**2))/\
                       Var)
            Geo.append(ar2)
        Sa_G_dic = create_dict(Geo,Sa_G)
        T_G_dic = create_dict(Geo,T_G)
        V.append(variables[imb][0])
        Sa_V.append(Sa_G_dic)
        T_V.append(T_G_dic)
    Sa_V_dic = create_dict(V,Sa_V)
    T_V_dic = create_dict(V,T_V)
    
    c_r.append(re)
    Sa_r.append(Sa_V_dic)
    T_r.append(T_V_dic)
Sa_r_dic = create_dict(c_r,Sa_r)
T_r_dic = create_dict(c_r,T_r)
Sa_of_df = {Sa_k: pd.DataFrame(Sa_v) for Sa_k,Sa_v in Sa_r_dic.items()}
T_of_df = {T_k: pd.DataFrame(T_v) for T_k,T_v in T_r_dic.items()}
T_df = pd.concat(T_of_df, axis=0)
Sa_df = pd.concat(Sa_of_df, axis=0)

### Irrigated Land Area Distribution plots

Charts of the irrigated land distributions can be generated as to evaluate their visual features at the continental level.

In [ ]:
co = ['b','r','g','k','c','m']
plt.style.use('ggplot')
for ar2 in Areas2:
    ax = sns.distplot(sM_A[0].Y[sM_A[0].Continent==ar2], bins=500, kde=False, axlabel='Irrigated Area 2050 (ha)_'+str(ar2))
    ax.set_xscale('log')
    ax.set_ylabel('Count')
    for ds in K:
        plt.axvline(x=K[ds][ar2],label=str(ds)+' Cropland in 2050')
    plt.legend()
    plt.show()

### Distribution plots World Area

The same plot can be reproduced at the global level by aggregating the continental figures.

In [ ]:
plt.style.use('ggplot')
ax = sns.distplot(worldIrrigatedArea, bins=100, kde=False, color='k',axlabel='Global Irrigated Area 2050 (ha)')
ax.set_xscale('log')
ax.set_ylabel('Count')
plt.fill_between((worldIrrigatedArea.quantile(.025),worldIrrigatedArea.quantile(.975)),(15000,15000),color='r',alpha=0.25,
                label='95% percentile')
plt.fill_between((2.4e8,4.5e8),(15000,15000),color='w',alpha=0.25,label='State of the art projections')
plt.yticks(np.arange(0, 1.5e4, step=5e3))
plt.ylim(0,1.5e4)
plt.legend()
plt.show()

### Scatter Plots

Scatter plots allow to visually convey the same information as sensitivity indices. A non-random trend of the output against an input parameter means this latter is influential the output uncertainty.

In [ ]:
co = ['Dataset','Regression','Robust','r','Y0','Epsilon','Year0','Gamma','K']
for ar2 in Areas2:
    for col in co:
        plt.figure(figsize=(20,10))
        plt.scatter(sM_As[0][col][sM_As[0].Continent==ar2],sM_As[0]['Y'][sM_As[0].Continent==ar2],s=1,label=str(ar2)+'_'+str(col))
        plt.yscale('log')
        plt.ylim(1e6,2e9)
        plt.legend()
        plt.show()

### Sensitivity-indices box plots

Whisker box plot of the sensitivity indices previously computed can be produced as to evaluate the effect of the input uncertainty on the irrigated area uncertainty. The indices are included in the range 0-1, the higher the index the more important the parameter. First order indices **S** tell us which fraction of the output variance a parameter is individually responsible for. Total order indices **T** describe what fraction of the output variance a parameter is responsible for also through interactions with the other parameters.

In [ ]:
for ar2 in Areas2:
    fig, ax = plt.subplots(figsize=(20,10))
    for iv,v in enumerate(variables):
        T_df.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='T', color='b',patch_artist=True)
        Sa_df.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='S',positions=[iv+0.5 for iv in range(len(variables))],
                                              color='r',patch_artist=True)
    m_patch = mpatches.Patch(color='b'
                             , label='T')
    c_patch = mpatches.Patch(color='r', label='S')
    plt.legend(handles=[m_patch,c_patch])
    plt.xticks([iv+0.75 for iv in range(len(variables))])
    plt.xlim(0,15.5)
    plt.title(ar2)
    plt.show()

## Defining variables for the sensitivity analysis of the cluster

The blocks of codes that follow repeat the same flows of calculation for the three categories of clusters: **irrigation, model, population**. The cells can be executed to visually reproduce the results reported in *Figure 2B* of the manuscript.

In [ ]:
variables3 = [['Dataset','Y0','Dataset_w','Wa','eta'],['Regression','Robust','X4','Robust_w','w4','Epsilon'],['r','Gamma','Year0','N0']]

In [ ]:
tr_As2 = [triggers] 
for tr2 in range(1,len(variables3)):
    tr_As2.append(triggers.copy())
    
tr_As_w2 = [triggers_w] 
for tr2 in range(1,len(variables3)):
    tr_As_w2.append(triggers_w.copy())

tr_As2[1][1]=triggersB[1]
tr_As2[-1][2:]=triggersB[2:]

tr_As_w2[1][1]=triggers_wB[1]
tr_As_w2[-1][2:]=triggers_wB[2:]

tr_Bs2 = [triggersB] 

tr_Bs_w2 = [triggers_wB]

tIndexA2 = [trigger_index(tri2.T) for tri2 in tr_As2]
tIndexA2_w = [trigger_index_water(tri2.T) for tri2 in tr_As_w2]

tIndexB2 = [trigger_index(tri2B.T) for tri2B in tr_Bs2]
tIndexB2_w = [trigger_index_water(tri2B.T) for tri2B in tr_Bs_w2]

In [ ]:
sM_As2 = []
for itI2,tI2 in enumerate(tIndexA2):
    sM_As2.append(lu_pop.copy())
    sM_As2[itI2]=sM_As2[itI2].reindex(tI2)
    sM_As2[itI2]=sM_As2[itI2].reset_index(drop=True)
    
sM_Bs2 = []
for itIB2,tIB2 in enumerate(tIndexB2):
    sM_Bs2.append(lu_pop.copy())
    sM_Bs2[itIB2]=sM_Bs2[itIB2].reindex(tIB2)
    sM_Bs2[itIB2]=sM_Bs2[itIB2].reset_index(drop=True)
        
sM_As_w2 = []
for itI,tI in enumerate(tIndexA2_w):
    sM_As_w2.append(lu_water.copy())
    sM_As_w2[itI]=sM_As_w2[itI].reindex(tI)
    sM_As_w2[itI]=sM_As_w2[itI].reset_index(drop=True)
    sM_As_w2[itI]=pd.concat([sM_As2[itI],sM_As_w2[itI]],axis=1)
    
sM_Bs_w2 = []
for itIB,tIB in enumerate(tIndexB2_w):
    sM_Bs_w2.append(lu_water.copy())
    sM_Bs_w2[itIB]=sM_Bs_w2[itIB].reindex(tIB)
    sM_Bs_w2[itIB]=sM_Bs_w2[itIB].reset_index(drop=True)
    sM_Bs_w2[itIB]=pd.concat([sM_Bs2[0],sM_Bs_w2[itIB]],axis=1)

In [ ]:
for sm2 in sM_As_w2:
    sm2['r']=p4.values
    sm2['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,7]-1),4)
    sm2['Year0'] = np.mean(PPop.columns).astype(int)
    sm2['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,8]-1)+1,4)
    sm2['Y0']=0
    sm2['K']=0
    sm2['Wa']=0
    sm2['eta']=np.tile(eta['Lower']+qM[:,13]*(eta['Upper']-eta['Lower']),4)
    for ar2 in Areas2:
        sm2.loc[sm2['Continent']==ar2,'Y0']=Y0.loc[ar2].min()+qM[:,10]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        sm2.loc[sm2['Continent']==ar2,'K']=K['Net'].loc[ar2]+qM[:,11]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
        sm2.loc[sm2['Continent']==ar2,'Wa']=Wa['Lower'].loc[ar2]+qM[:,12]*(Wa['Upper'].loc[ar2]-Wa['Lower'].loc[ar2])
    sm2['N0']=PPop[np.mean(PPop.columns).astype(int)].loc[ar2]
        
for smB2 in sM_Bs_w2:
    smB2['r']=p4B.values
    smB2['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,21]-1),4)
    smB2['Year0'] = np.mean(PPop.columns).astype(int)
    smB2['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,22]-1)+1,4)
    smB2['Y0']=0
    smB2['K']=0
    smB2['Wa']=0
    smB2['eta']=np.tile(eta['Lower']+qM[:,27]*(eta['Upper']-eta['Lower']),4)
    for ar2 in Areas2:
        smB2.loc[smB2['Continent']==ar2,'Y0']=Y0.loc[ar2].min()+qM[:,24]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        smB2.loc[smB2['Continent']==ar2,'K']=K['Net'].loc[ar2]+qM[:,25]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
        smB2.loc[smB2['Continent']==ar2,'Wa']=Wa['Lower'].loc[ar2]+qM[:,26]*(Wa['Upper'].loc[ar2]-Wa['Lower'].loc[ar2])
    smB2['N0']=PPop[np.mean(PPop.columns).astype(int)].loc[ar2]

### Generate the scrambled matrices

In [ ]:
sM_As_w2.append(sM_As_w2[0].copy())

sM_As_w2[1][['Y0','Wa','eta']]=sM_Bs_w2[0][['Y0','Wa','eta']]
sM_As_w2[2]['Epsilon']=sM_Bs_w2[0]['Epsilon']
sM_As_w2[-1][['Gamma','r','Year0','N0']]=sM_Bs_w2[0][['Gamma','r','Year0','N0']]

### Calculate Y and replace overshooting Y values with K figures

In [ ]:
for sm2 in sM_As_w2:    
    sm2['Y']=Y(sm2.Y0,sm2.Beta,sm2.Epsilon,sm2.r,sm2.N0,sm2.Gamma,sm2.Year0)
    sm2.loc[sm2['Y']>sm2.K,'Y'] = sm2.K
    sm2.loc[(sm2.Wa*sm2.eta<sm2.Y**sm2.Delta*10**sm2.Phi)&((sm2.Wa*sm2.eta/10**sm2.Phi)**(1/sm2.Delta) > sm2.Y0),'Y'] = \
    (sm2.Wa*sm2.eta/10**sm2.Phi)**(1/10**sm2.Delta)
    sm2['Y'].loc[(sm2.Wa*sm2.eta<sm2.Y**sm2.Delta*10**sm2.Phi)&((sm2.Wa*sm2.eta/10**sm2.Phi)**(1/sm2.Delta) < sm2.Y0)] = sm2.Y0
    
for smB2 in sM_Bs_w2:
    smB2['Y']=Y(smB2.Y0,smB2.Beta,smB2.Epsilon,smB2.r,smB2.N0,smB2.Gamma,smB2.Year0)
    smB2.loc[smB2['Y']>smB2.K,'Y'] = smB2.K
    smB2.loc[(smB2.Wa*smB2.eta<smB2.Y**smB2.Delta*10**smB2.Phi)&((smB2.Wa*smB2.eta/10**smB2.Phi)**(1/smB2.Delta) >  
                                                        smB2.Y0),'Y'] = (smB2.Wa*smB2.eta/10**smB2.Phi)**(1/smB2.Delta)
    smB2.loc[(smB2.Wa*smB2.eta<smB2.Y**smB2.Delta*10**smB2.Phi)&((smB2.Wa*smB2.eta/10**smB2.Phi)**(1/smB2.Delta) <  
                                                                            smB2.Y0),'Y'] = smB2.Y0

### Filter out negative values

In [ ]:
sM_As_w2 = [sm2[sm2['Y']>0] for sm2 in sM_As_w2]

sM_Bs_w2 = [smB2[smB2['Y']>0] for smB2 in sM_Bs_w2]

In [ ]:
sM_S2 = sM_As_w2.copy()
sM_S2.append(sM_Bs_w2[0])

In [ ]:
inr2 = sM_S2[0].index.intersection(sM_S2[1].index).intersection(sM_S2[2].index).intersection(sM_S2[3].index).intersection(sM_S2[4].index)

In [ ]:
sM_S2 = [sms2.merge(pd.DataFrame(index=inr2), left_index=True, right_index=True, how='right') for sms2 in sM_S2]

In [ ]:
c_r2 = []
T_r2 = []
Sa_r2 = []
for re in range(1000):    
    seed = (qMbS[re*len(sM_S2[0]):(re+1)*len(sM_S2[0])]*len(sM_S2[0])).astype(int)
    
    sM_BS2 = [sms2.merge(pd.DataFrame(index=seed), left_index=True, right_index=True, how='right') for sms2 in sM_S2]

    V = []
    Sa_V = []
    T_V = []
    for imb,mb in enumerate(sM_BS2[1:4]):
        Geo = []
        Sa_G = []
        T_G = []
        for ar2 in Areas2:
            Sa_G.append(((mb['Y'][mb['Continent']==ar2]-sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2])*
            sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2]).mean()/\
            pd.concat([sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2],sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2]]).var(ddof=0))
            Geo.append(ar2)
            T_G.append(0.5*((mb['Y'][mb['Continent']==ar2]-sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2])**2).mean()/\
                pd.concat([sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2],sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2]]).var(ddof=0))
        Sa_G_dic = create_dict(Geo,Sa_G)
        T_G_dic = create_dict(Geo,T_G)
        V.append(variables3[imb][0])
        Sa_V.append(Sa_G_dic)
        T_V.append(T_G_dic)
    Sa_V_dic = create_dict(V,Sa_V)
    T_V_dic = create_dict(V,T_V)
    
    c_r2.append(re)
    Sa_r2.append(Sa_V_dic)
    T_r2.append(T_V_dic)
    
Sa_r_dic2 = create_dict(c_r2,Sa_r2)
T_r_dic2 = create_dict(c_r2,T_r2)
Sa_of_df2 = {Sa_k2: pd.DataFrame(Sa_v2) for Sa_k2,Sa_v2 in Sa_r_dic2.items()}
T_of_df2 = {T_k2: pd.DataFrame(T_v2) for T_k2,T_v2 in T_r_dic2.items()}
T_df2 = pd.concat(T_of_df2, axis=0)
Sa_df2 = pd.concat(Sa_of_df2, axis=0)

### Sensitivity indices box plots

In [ ]:
T_df2 = T_df2.rename(columns={'Dataset':'Irrigation','Regression':'Model','r':'Population'})
Sa_df2 = Sa_df2.rename(columns={'Dataset':'Irrigation','Regression':'Model','r':'Population'})
for ar2 in Areas2:
    fig, ax = plt.subplots(figsize=(20,10))
    for iv,v in enumerate(variables3):
        T_df2.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='T', color='b',patch_artist=True)
        Sa_df2.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='S',positions=[iv+0.5 for iv in range(len(variables3))],
                                              color='r',patch_artist=True)
    m_patch = mpatches.Patch(color='b', label='T')
    c_patch = mpatches.Patch(color='r', label='S')
    plt.legend(handles=[m_patch,c_patch])
    plt.xticks([iv+0.75 for iv in range(len(variables3))])
    plt.xlim(0,3.5)
    plt.title(ar2)
    plt.show()